# Read and Cut HiHydroSoil and SoilGrids data for JULES pre-processing
Domain 3 (Chile and parts of Argentina) and domain 4 (Peru and Bolivia)



Connect to Google Earth Engine

In [ ]:
# import Google Earth Engine API
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='ee-rikebecker')

Import libraries

In [ ]:
import folium
from folium import plugins
import time

Import data

In [ ]:
# import global soil data data available in the google earth engine data catalog
# global soil data sets in 250m resolution

#HiHydroSoil v2.0 data on GEE: https://gee-community-catalog.org/projects/hihydro_soil/
Sm_crit = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcpf3")
Sm_sat = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcsat")
Sm_wilt = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcpf4-2")
Ksat = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/ksat")
Alpha = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/alpha")
N = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/N")

#SoilGrids v2.0 data on GEE: https://gee-community-catalog.org/projects/isric/
sand = ee.Image("projects/soilgrids-isric/sand_mean")
silt = ee.Image("projects/soilgrids-isric/silt_mean")
clay = ee.Image("projects/soilgrids-isric/clay_mean")
cec = ee.Image("projects/soilgrids-isric/cec_mean") # cation excahange capacities
soc = ee.Image("projects/soilgrids-isric/soc_mean") # soil organic carbon
pH = ee.Image("projects/soilgrids-isric/phh2o_mean")# pH
bd = ee.Image("projects/soilgrids-isric/bdod_mean")# bulk density

Cut images for areas slighty larger than the Deplete and Retreat WRF domains

In [ ]:
# Define the bounding box coordinates
# Domain 03
minLon = -80
minLat = -57
maxLon = -61.5
maxLat = -15
bbox_03 = ee.Geometry.Rectangle([minLon, minLat, maxLon, maxLat])

In [ ]:
# Domain 04
minLon = -84.5
minLat = -21.8
maxLon = -62.0
maxLat = -2.8
bbox_04 = ee.Geometry.Rectangle([minLon, minLat, maxLon, maxLat])

**Important !!! Set the domain you want to process the data for**

In [ ]:
#bbox = bbox_03
bbox = bbox_04

#domain = 'd03'
domain = 'd04'

Clip the data to the size of the domain

In [ ]:
# Function to clip each image in the collection
def clip_image(image):
    return image.clip(bbox)

In [ ]:
# Clip each image in the collection
Sm_crit = Sm_crit.map(clip_image)
Sm_sat = Sm_sat.map(clip_image)
Sm_wilt = Sm_wilt.map(clip_image)
Ksat = Ksat.map(clip_image)
Alpha = Alpha.map(clip_image)
N = N.map(clip_image)

sand = sand.clip(bbox)
silt = silt.clip(bbox)
clay = clay.clip(bbox)
cec = cec.clip(bbox)
soc = soc.clip(bbox)
pH = pH.clip(bbox)
bd = bd.clip(bbox)

# Check the size of the clipped collection
#print("Number of images in clipped collection:", Sm_crit.size().getInfo())

Plot to check the results

In [ ]:
# Get a URL that displays a map of the clipped image
map_id = sand.getMapId()

# Create a map centered at the bounding box
center_lat = (minLat + maxLat) / 2
center_lon = (minLon + maxLon) / 2
map_center = [center_lat, center_lon]
map = folium.Map(location=map_center, zoom_start=10)

# Add the clipped image as a tile layer to the map
folium.TileLayer(
    tiles=map_id['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='Clipped Image'
).add_to(map)

# Add the bounding box to the map as a rectangle
folium.Rectangle(
    bounds=[[minLat, minLon], [maxLat, maxLon]],
    color='red',
    fill=False
).add_to(map)

# Display the map
display(map)


Save images to local disc

Sm_crit

In [ ]:
# Load the image collection you want to export
clippedCollection = Sm_crit

# Export each image in the clipped collection to Google Drive
for i in range(clippedCollection.size().getInfo()):
    image = ee.Image(clippedCollection.toList(clippedCollection.size()).get(i))

    # Start the export task for each image
    task = ee.batch.Export.image.toDrive(
        image = image,
        description = 'Sm_crit',  # Name of the output asset
        folder = 'earth_engine_exports',  # Name of the Google Drive folder
        fileNamePrefix = 'Sm_crit_'+ domain + '_' + str(i),
        region = bbox,  # Region to export
        scale = 1000,  # Scale in meters
        crs = image.projection(),
        fileFormat = 'GeoTIFF')

    task.start()

    # Print a message
    print("Exporting image", i)

    # Wait for the export to finish
    while task.active():
        print("Exporting...")
        time.sleep(30)

print("Export completed.")

Exporting image 0
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Exporting image 1
Exporting...


KeyboardInterrupt: 

Sm_sat

In [ ]:
# Load the image collection you want to export
clippedCollection = Sm_sat

# Export each image in the clipped collection to Google Drive
for i in range(clippedCollection.size().getInfo()):
    image = ee.Image(clippedCollection.toList(clippedCollection.size()).get(i))

    # Start the export task for each image
    task = ee.batch.Export.image.toDrive(
        image = image,
        description = 'Sm_sat',  # Name of the output asset
        folder = 'earth_engine_exports',  # Name of the Google Drive folder
        fileNamePrefix = 'Sm_sat' + domain + '_' + str(i),  # Prefix of the output files
        region = bbox,  # Region to export
        scale = 1000,  # Scale in meters
        fileFormat = 'GeoTIFF')

    task.start()

    # Print a message
    print("Exporting image", i)

    # Wait for the export to finish
    while task.active():
        print("Exporting...")
        time.sleep(30)

print("Export completed.")

Sm_wilt

In [ ]:
# Load the image collection you want to export
clippedCollection = Sm_wilt

# Export each image in the clipped collection to Google Drive
for i in range(clippedCollection.size().getInfo()):
    image = ee.Image(clippedCollection.toList(clippedCollection.size()).get(i))

    # Start the export task for each image
    task = ee.batch.Export.image.toDrive(
        image = image,
        description = 'Sm_wilt',  # Name of the output asset
        folder = 'earth_engine_exports',  # Name of the Google Drive folder
        fileNamePrefix = 'Sm_wilt_'+ domain + '_' + str(i),  # Prefix of the output files
        region = bbox,  # Region to export
        scale = 1000,  # Scale in meters
        fileFormat = 'GeoTIFF')

    task.start()

    # Print a message
    print("Exporting image", i)

    # Wait for the export to finish
    while task.active():
        print("Exporting...")
        time.sleep(30)

print("Export completed.")

Ksat

In [ ]:
# Load the image collection you want to export
clippedCollection = Ksat

# Export each image in the clipped collection to Google Drive
for i in range(clippedCollection.size().getInfo()):
    image = ee.Image(clippedCollection.toList(clippedCollection.size()).get(i))

    # Start the export task for each image
    task = ee.batch.Export.image.toDrive(
        image = image,
        description = 'Ksat',  # Name of the output asset
        folder = 'earth_engine_exports',  # Name of the Google Drive folder
        fileNamePrefix = 'Ksat_' + domain + '_' + str(i),  # Prefix of the output files
        region = bbox,  # Region to export
        scale = 1000,  # Scale in meters
        fileFormat = 'GeoTIFF')

    task.start()

    # Print a message
    print("Exporting image", i)

    # Wait for the export to finish
    while task.active():
        print("Exporting...")
        time.sleep(30)

print("Export completed.")

Exporting image 0
Exporting...


KeyboardInterrupt: 

Alpha

In [ ]:
# Load the image collection you want to export
clippedCollection = Alpha

# Export each image in the clipped collection to Google Drive
for i in range(clippedCollection.size().getInfo()):
    image = ee.Image(clippedCollection.toList(clippedCollection.size()).get(i))

    # Start the export task for each image
    task = ee.batch.Export.image.toDrive(
        image = image,
        description = 'Alpha',  # Name of the output asset
        folder = 'earth_engine_exports',  # Name of the Google Drive folder
        fileNamePrefix = 'Alpha_' + domain + '_' + str(i),  # Prefix of the output files
        region = bbox,  # Region to export
        scale = 1000,  # Scale in meters
        fileFormat = 'GeoTIFF')

    task.start()

    # Print a message
    print("Exporting image", i)

    # Wait for the export to finish
    while task.active():
        print("Exporting...")
        time.sleep(30)

print("Export completed.")

Exporting image 0
Exporting...
Exporting...
Exporting...
Exporting...
Exporting image 1
Exporting...
Exporting...
Exporting...
Exporting...


N - Van Genuchten parameter

In [ ]:
# Load the image collection you want to export
clippedCollection = N

# Export each image in the clipped collection to Google Drive
for i in range(clippedCollection.size().getInfo()):
    image = ee.Image(clippedCollection.toList(clippedCollection.size()).get(i))

    # Start the export task for each image
    task = ee.batch.Export.image.toDrive(
        image = image,
        description = 'N',  # Name of the output asset
        folder = 'earth_engine_exports',  # Name of the Google Drive folder
        fileNamePrefix = 'N_' + domain + '_' + str(i),  # Prefix of the output files
        region = bbox,  # Region to export
        scale = 1000,  # Scale in meters
        fileFormat = 'GeoTIFF')

    task.start()

    # Print a message
    print("Exporting image", i)

    # Wait for the export to finish
    while task.active():
        print("Exporting...")
        time.sleep(30)

print("Export completed.")

Sand

In [ ]:
# Load the image you want to export
image_to_export = sand
reference_image = Sm_crit.first()
reference_projection = reference_image.projection() # Get the projection of the reference image
reference_scale = reference_projection.nominalScale().getInfo() # Get the nominal scale of the reference projection

# Reproject the image to match the resolution of the reference image
reprojected_image = image_to_export.reproject(crs=reference_projection, scale=reference_scale)


# Define the export parameters
export_params = {
    'image': reprojected_image,
    'description': 'Sand_'+ domain,
    'folder': 'earth_engine_exports',
    'scale': reference_scale,  # Use the nominal scale of the reference image
    'region': bbox,  # Set the region to match the reference image
    'crs': reference_projection.crs().getInfo(),  # Set the CRS to match the reference image
    'maxPixels': 1e13,  # Increase maxPixels is necessary otherwise it fails to export
}

# Export the image
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Optional: Monitor the task
import time
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(30)
print('Task status:', task.status())

Polling for task (id: P6XF2FHSMPI4TI4QYQTUZIGI).


KeyboardInterrupt: 

Silt

In [ ]:
# Load the image collection you want to export
image_to_export = silt
reference_image = Sm_crit.first()

# Get the projection of the reference image
reference_projection = reference_image.projection()

# Define the export parameters
export_params = {
    'image': image_to_export.reproject(reference_projection),
    'description': 'Silt'+domain,
    'folder': 'earth_engine_exports',
    'scale': reference_projection.nominalScale().getInfo(),  # Use the nominal scale of the reference image
    'region': bbox,  # Set the region to match the reference image
    'crs': reference_projection.crs().getInfo(),  # Set the CRS to match the reference image
    'maxPixels': 1e13,  # Increase maxPixels is necessary otherwise it fails to export
}

# Export the image
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Optional: Monitor the task
import time
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(30)
print('Task status:', task.status())


Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id: IPESOGLYELLRHFRIGIRNUBBH).
Polling for task (id

Clay

In [ ]:
# Load the image collection you want to export
image_to_export = clay
reference_image = Sm_crit.first()

# Get the projection of the reference image
reference_projection = reference_image.projection()

# Define the export parameters
export_params = {
    'image': image_to_export.reproject(reference_projection),
    'description': 'Clay'+domain,
    'folder': 'earth_engine_exports',
    'scale': reference_projection.nominalScale().getInfo(),  # Use the nominal scale of the reference image
    'region': bbox,  # Set the region to match the reference image
    'crs': reference_projection.crs().getInfo(),  # Set the CRS to match the reference image
    'maxPixels': 1e13,  # Increase maxPixels is necessary otherwise it fails to export
}

# Export the image
task = ee.batch.Export.image.toDrive(**export_params)
task.start()


# Optional: Monitor the task
import time
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(30)
print('Task status:', task.status())

task_status = task.status()
print('Task status:', task_status)
if task_status['state'] == 'FAILED':
    print('Error message:', task_status['error_message'])

Exporting image
Exporting...
Exporting...
Exporting...
Exporting...
Exporting...
Export completed.


Soil organic carbon (soc)

In [1]:
# Load the image collection you want to export
image_to_export = soc
reference_image = Sm_crit.first()

# Get the projection of the reference image
reference_projection = reference_image.projection()

# Define the export parameters
export_params = {
    'image': image_to_export.reproject(reference_projection),
    'description': 'soc'+domain,
    'folder': 'earth_engine_exports',
    'scale': reference_projection.nominalScale().getInfo(),  # Use the nominal scale of the reference image
    'region': bbox,  # Set the region to match the reference image
    'crs': reference_projection.crs().getInfo(),  # Set the CRS to match the reference image
    'maxPixels': 1e13,  # Increase maxPixels is necessary otherwise it fails to export
}

# Export the image
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Optional: Monitor the task
import time
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(30)
print('Task status:', task.status())

task_status = task.status()
print('Task status:', task_status)
if task_status['state'] == 'FAILED':
    print('Error message:', task_status['error_message'])

NameError: name 'soc' is not defined

Cation exchange capacity (cec)

In [ ]:
# Load the image collection you want to export
image_to_export = cec
reference_image = Sm_crit.first()

# Get the projection of the reference image
reference_projection = reference_image.projection()

# Define the export parameters
export_params = {
    'image': image_to_export.reproject(reference_projection),
    'description': 'cec'+domain,
    'folder': 'earth_engine_exports',
    'scale': reference_projection.nominalScale().getInfo(),  # Use the nominal scale of the reference image
    'region': bbox,  # Set the region to match the reference image
    'crs': reference_projection.crs().getInfo(),  # Set the CRS to match the reference image
    'maxPixels': 1e13,  # Increase maxPixels is necessary otherwise it fails to export
}

# Export the image
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Optional: Monitor the task
import time
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(30)
print('Task status:', task.status())

task_status = task.status()
print('Task status:', task_status)
if task_status['state'] == 'FAILED':
    print('Error message:', task_status['error_message'])

Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Polling for task (id: VR55QVBH24XY3ZJNESTFQV24).
Task status: {'state': 'COMPLETED', 'description': 'cec_d03', 'priority': 100, 'creation_timestamp_ms': 1718294485533,

pH - Index

In [ ]:
# Load the image collection you want to export
image_to_export = pH
reference_image = Sm_crit.first()

# Get the projection of the reference image
reference_projection = reference_image.projection()

# Define the export parameters
export_params = {
    'image': image_to_export.reproject(reference_projection),
    'description': 'pH'+domain,
    'folder': 'earth_engine_exports',
    'scale': reference_projection.nominalScale().getInfo(),  # Use the nominal scale of the reference image
    'region': bbox,  # Set the region to match the reference image
    'crs': reference_projection.crs().getInfo(),  # Set the CRS to match the reference image
    'maxPixels': 1e13,  # Increase maxPixels is necessary otherwise it fails to export
}

# Export the image
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Optional: Monitor the task
import time
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(30)
print('Task status:', task.status())

task_status = task.status()
print('Task status:', task_status)
if task_status['state'] == 'FAILED':
    print('Error message:', task_status['error_message'])

Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id: IZRPZ5DUESLXLGCC3ZAHAFJR).
Polling for task (id

Bulk Density

In [ ]:
# Load the image collection you want to export
image_to_export = bd
reference_image = Sm_crit.first()

# Get the projection of the reference image
reference_projection = reference_image.projection()

# Define the export parameters
export_params = {
    'image': image_to_export.reproject(reference_projection),
    'description': 'bd_'+domain,
    'folder': 'earth_engine_exports',
    'scale': reference_projection.nominalScale().getInfo(),  # Use the nominal scale of the reference image
    'region': bbox,  # Set the region to match the reference image
    'crs': reference_projection.crs().getInfo(),  # Set the CRS to match the reference image
    'maxPixels': 1e13,  # Increase maxPixels is necessary otherwise it fails to export
}

# Export the image
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Optional: Monitor the task
import time
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(30)

task_status = task.status()
print('Task status:', task_status)
if task_status['state'] == 'FAILED':
    print('Error message:', task_status['error_message'])